In [ ]:
import tensorflow.compat.v1 as tf

tf.disable_v2_behavior()
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import numpy as np

In [ ]:
#Loading and splitting the data
allData = pd.read_csv("data/songs_with_genre.csv", index_col=0,
                      header=[0, 1, 2])
generes = allData["genre_id"]
genres_index = pd.read_csv("data/genres_index.csv", index_col=0)
allData.drop(columns=["genre_id"], inplace=True)
data_x = allData
data_x.columns = range(data_x.shape[1])

In [ ]:
mu = np.mean(data_x, axis=0)
sigma = (np.std(data_x, axis=0))
data_x = (data_x - mu) / sigma

In [ ]:
#Creating the right lables
generes.columns = range(generes.shape[1])
targets = OneHotEncoder().fit_transform(genres_index).toarray()
generes[0] = generes[0] - 1
data_y = [targets[i - 1] for i in generes[0]]
generes[0] = data_y

In [ ]:
test = generes.copy()
test.rename(columns={0: "oneHotVector"}, inplace=True)
test.shape

In [ ]:
split = pd.DataFrame(test["oneHotVector"].to_list(), columns=range(161))
print(split.shape)
split.head()

In [ ]:
split.index = generes.index

In [ ]:
generes = split

In [ ]:
#Creating train,test and validation sets
x_train, x_rest, y_train, y_rest = train_test_split(data_x, generes, train_size=0.8)
x_test, x_valid, y_test, y_valid = train_test_split(x_rest, y_rest, train_size=0.5)

In [ ]:
x_train.head()

In [ ]:
#Defining the initial weights and input/output

(first_layer_size, second_layer_size, third_layer_size) = [518 // 8, 518 // 4, 518 // 8]
x = tf.placeholder(tf.float32, [None, 518])
y = tf.placeholder(tf.float32, [None, 161])

weight_first = tf.Variable(tf.truncated_normal([518, first_layer_size], stddev=0.3, dtype=tf.float32))
bias_first = tf.Variable(tf.constant(0, shape=[first_layer_size], dtype=tf.float32))
activation_first = tf.nn.relu6(tf.matmul(x, weight_first) + bias_first)

weight_second = tf.Variable(tf.truncated_normal([first_layer_size, second_layer_size], stddev=0.3, dtype=tf.float32))
bias_second = tf.Variable(tf.constant(0, shape=[second_layer_size], dtype=tf.float32))
activation_second = tf.nn.relu6(tf.matmul(activation_first, weight_second) + bias_second)

weight_third = tf.Variable(tf.truncated_normal([second_layer_size, third_layer_size], stddev=0.3, dtype=tf.float32))
bias_third = tf.Variable(tf.constant(0, shape=[third_layer_size], dtype=tf.float32))
activation_third = tf.nn.relu6(tf.matmul(activation_second, weight_third) + bias_third)

weights_output = tf.Variable(tf.truncated_normal([third_layer_size, 161], stddev=0.3, dtype=tf.float32))
bias_output = tf.Variable(tf.constant(0, shape=[161], dtype=tf.float32))


In [ ]:
#creating prediction, training and cross-entropy methods
pred = tf.nn.softmax(tf.matmul(activation_third, weights_output) + bias_output)
loss = -tf.reduce_sum(y * tf.log(tf.clip_by_value(pred, 1e-10, 1.0)))
training_step = tf.train.AdamOptimizer(0.01).minimize(loss)

In [ ]:
#setting session and starting to run
training_sess = tf.Session()
training_sess.run(tf.global_variables_initializer())

In [ ]:
#setting function for correct prediction and accuracy
correct = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [ ]:
#training loop:
saver = tf.train.Saver()
epochs = 10001
batch_size = 389
iterations = int((x_train.shape[0]) / batch_size)
maxAccuracy = 0
for epoch in range(epochs):
    ptr = 0
    for iteration in range(iterations):
        batch_input = x_train[ptr:ptr + batch_size]
        batch_label = y_train[ptr:ptr + batch_size]
        ptr = (ptr + batch_size) % x_train.shape[0]
        _, err = training_sess.run([training_step, loss], feed_dict={x: batch_input, y: batch_label})
    if epoch % 1000 == 0:
        currAccuracy = training_sess.run(accuracy, feed_dict={x: x_train, y: y_train})
        valid_accuracy = training_sess.run(accuracy, feed_dict={x: x_valid, y: y_valid})
        if currAccuracy > maxAccuracy:
            saver.save(training_sess, "best_mlp_7_layers", global_step=1)
            maxAccuracy = currAccuracy
        print(f"epoch: {epoch} - accuracy for epoch {currAccuracy*100} - max accuracy: {maxAccuracy*100}")
        print(f"validation set accuracy: {valid_accuracy*100}")

In [ ]:
x_train.shape

In [ ]:
# test_accuracy = training_sess.run(accuracy, feed_dict={x: x_test, y: y_test})
# print(test_accuracy)